In [2]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(URL(
        account='to57169',
        user='yang',
        password='yang123!',
        warehouse = 'fivetran_warehouse',
        database='fivetran',
        schema='bbbnw'

    )
)

In [3]:
master= pd.read_sql_query('DESC TABLE master', engine)
print(master)

                    name               type    kind null? default primary key  \
0            BUSINESS_ID       NUMBER(38,0)  COLUMN     Y    None           N   
1                  TR_ID       NUMBER(38,0)  COLUMN     Y    None           N   
2                   TYPE  VARCHAR(16777216)  COLUMN     Y    None           N   
3               BUSINESS  VARCHAR(16777216)  COLUMN     Y    None           N   
4              ADDRESS_1  VARCHAR(16777216)  COLUMN     Y    None           N   
5              ADDRESS_2  VARCHAR(16777216)  COLUMN     Y    None           N   
6                   CITY  VARCHAR(16777216)  COLUMN     Y    None           N   
7                  STATE  VARCHAR(16777216)  COLUMN     Y    None           N   
8                    ZIP       NUMBER(38,0)  COLUMN     Y    None           N   
9                  ZIP_4       NUMBER(38,0)  COLUMN     Y    None           N   
10         PRIMARY_PHONE  VARCHAR(16777216)  COLUMN     Y    None           N   
11              INQ_3_YR    

In [4]:
businesslicense = pd.read_sql_query('DESC TABLE businesslicense', engine)
print(businesslicense)

                           name               type    kind null? default  \
0                 LICENSENUMBER       NUMBER(38,0)  COLUMN     Y    None   
1                  BUSINESSNAME  VARCHAR(16777216)  COLUMN     Y    None   
2                        STATUS  VARCHAR(16777216)  COLUMN     Y    None   
3                  BUSINESSTYPE  VARCHAR(16777216)  COLUMN     Y    None   
4                OWNERFIRSTNAME  VARCHAR(16777216)  COLUMN     Y    None   
5       OWNERLASTORBUSINESSNAME  VARCHAR(16777216)  COLUMN     Y    None   
6                     ISSUEDATE  VARCHAR(16777216)  COLUMN     Y    None   
7                     RENEWDATE               DATE  COLUMN     Y    None   
8                    EXPIREDATE               DATE  COLUMN     Y    None   
9     PRIMARYLINEOFBUSINESSTEXT  VARCHAR(16777216)  COLUMN     Y    None   
10             PRIMARYNAICSTEXT  VARCHAR(16777216)  COLUMN     Y    None   
11  SECONDARYLINEOFBUSINESSTEXT  VARCHAR(16777216)  COLUMN     Y    None   
12          

In [5]:
professional = pd.read_sql_query('DESC TABLE professionallicense', engine)
print(professional)

             name               type    kind null? default primary key  \
0         PROGRAM  VARCHAR(16777216)  COLUMN     Y    None           N   
1       PROF_TYPE  VARCHAR(16777216)  COLUMN     Y    None           N   
2      LICENSENUM  VARCHAR(16777216)  COLUMN     Y    None           N   
3             DBA  VARCHAR(16777216)  COLUMN     Y    None           N   
4          OWNERS  VARCHAR(16777216)  COLUMN     Y    None           N   
5          STATUS  VARCHAR(16777216)  COLUMN     Y    None           N   
6      DATEISSUED               DATE  COLUMN     Y    None           N   
7   DATEEFFECTIVE               DATE  COLUMN     Y    None           N   
8     DATEEXPIRED               DATE  COLUMN     Y    None           N   
9       ADDRESS_1  VARCHAR(16777216)  COLUMN     Y    None           N   
10      ADDRESS_2  VARCHAR(16777216)  COLUMN     Y    None           N   
11           CITY  VARCHAR(16777216)  COLUMN     Y    None           N   
12          STATE  VARCHAR(16777216)  

In [6]:
active = pd.read_sql_query('DESC TABLE activeorbusinesses', engine)
print(active)

                    name               type    kind null? default primary key  \
0        REGISTRY_NUMBER       NUMBER(38,0)  COLUMN     Y    None           N   
1          BUSINESS_NAME  VARCHAR(16777216)  COLUMN     Y    None           N   
2                ADDRESS  VARCHAR(16777216)  COLUMN     Y    None           N   
3                   CITY  VARCHAR(16777216)  COLUMN     Y    None           N   
4                  STATE  VARCHAR(16777216)  COLUMN     Y    None           N   
5                    ZIP       NUMBER(38,0)  COLUMN     Y    None           N   
6            ENTITY_TYPE  VARCHAR(16777216)  COLUMN     Y    None           N   
7          REGISTRY_DATE  VARCHAR(16777216)  COLUMN     Y    None           N   
8   ASSOCIATED_NAME_TYPE  VARCHAR(16777216)  COLUMN     Y    None           N   
9             FIRST_NAME  VARCHAR(16777216)  COLUMN     Y    None           N   
10           MIDDLE_NAME  VARCHAR(16777216)  COLUMN     Y    None           N   
11     ADDRESS_CONTINUED  VA

In [8]:
#features_to_use = ["BUSINESS","STATE"， "ZIP”]
#Read businessname, state, and zipcode from master and 3rd party dataset, excluding all empty ZIP codes. 
master_bus_st = pd.read_sql_query('SELECT business, state, left(zip, 5) as zip FROM master WHERE zip IS NOT NULL', engine)
business_bus_st = pd.read_sql_query('SELECT businessname as business, mailingstatecode as state, left(mailingzip, 5) as zip FROM businesslicense WHERE mailingzip IS NOT NULL', engine)
#right_join_master_business = pd.read_sql_query('SELECT master.business, master.state, master.zip FROM master RIGHT JOIN businesslicense ON master.business = businesslicense.businessname WHERE master.business IS NULL AND businesslicense.mailingzip IS NOT NULL', engine)

In [9]:
#Taking the inner joins to exclude from computation
inner_join_master_active = pd.read_sql_query('SELECT master.business, master.state, master.zip FROM master INNER JOIN activeorbusinesses ON master.business = activeorbusinesses.business_name', engine)
inner_join_master_professional = pd.read_sql_query('SELECT master.business, master.state, master.zip FROM master INNER JOIN professionallicense ON master.business = professionallicense.owners', engine) 
inner_join_master_business = pd.read_sql_query('SELECT master.business, master.state, master.zip FROM master INNER JOIN businesslicense ON master.business = businesslicense.businessname', engine)

In [10]:
print(inner_join_master_business)
print(inner_join_master_professional)
print(inner_join_master_active)
print(right_join_master_business)

                            business state      zip
0                   A&J GIFTS & MORE    AK  99648.0
1               SWEET FACE ESTHETICS    AK  99504.0
2                     SELECT RENTALS    AK  99504.0
3            STOP N' PLAY CHILD CARE    AK  99504.0
4         O.C.D. CLEANING & SERVICES    AK  99502.0
5              ARTHUN PAINTING, INC.    AK  99518.0
6                   TECHTONIC CYCLES            NaN
7                  HANDYMAN SERVICES    WA  98934.0
8                      907 ROCK AUTO    AK  99521.0
9                      907 ROCK AUTO    AK  99521.0
10                     907 ROCK AUTO    AK  99521.0
11                     907 ROCK AUTO    AK  99521.0
12                              AJ'S    WA  98023.0
13                              AJ'S    WA  98023.0
14                              AJ'S    WA  98023.0
15                       HOUSE CALLS            NaN
16                       A CUT ABOVE            NaN
17     ALASKAS TRAIL KING ADVENTURES            NaN
18          

In [24]:
professional_bus_st = pd.read_sql_query('SELECT owners, state, left(zip, 5) as zip FROM professionallicense WHERE zip IS NOT NULL', engine)
print(professional_bus_st)

                                               owners state    zip
0                   Michael Baker International, Inc.    AK  99503
1                             R & M CONSULTANTS, INC.    AK  99507
2                   AECOM Energy & Construction, Inc.    ID  83642
3                                      AMEC E&I, INC.    GA  30004
4                                       BECHTEL, INC.    CA  94119
5                              SHANNON & WILSON, INC.    NY  98103
6                    Parsons Government Services Inc.    CA  91124
7                             JENSEN YORBA LOTT, INC.    AK  99801
8              TRAVIS, FITZMAURICE & ASSOCIATES, INC.    WA  98109
9                            SIMPSON ASSOCIATES, INC.    AK  99524
10                                          GDM, INC.    AK  99503
11                              POOL ENGINEERING INC.    AK  99901
12                                         USKH, INC.    AK  99503
13                             LIVINGSTONE SLONE INC.    AK  9

In [28]:
print (master_bus_st)
print (business_bus_st)

                                       business state    zip
0                              Wells Fargo Bank    OR  97201
1               Advance Central Services Oregon    OR  97201
2                  Finley Sunset Hills Mortuary    OR  97225
3                               Atiyeh Bros Inc    OR  97224
4                           Jantzen Apparel LLC    OR  97232
5                       Braley & Graham Company    OR  97225
6                                      JCPenney    OR  97223
7                   Wentworth Chevrolet Company    OR  97214
8          Lincoln Memorial Park & Funeral Home    OR  97086
9             Portland General Electric Company    OR  97208
10                                      Safeway    OR  97015
11                          Rodda Paint Company    OR  97203
12                Paul Schatz Furniture Company    OR  97224
13                    Bonded Adjustment Company    WA  99201
14                        K X L AM & K X J M FM    OR  97209
15                      

In [29]:
print (right_join_master_business - inner_join_master_business)

      business state  zip
0          NaN   NaN  NaN
1          NaN   NaN  NaN
2          NaN   NaN  NaN
3          NaN   NaN  NaN
4          NaN   NaN  NaN
5          NaN   NaN  NaN
6          NaN   NaN  NaN
7          NaN   NaN  NaN
8          NaN   NaN  NaN
9          NaN   NaN  NaN
10         NaN   NaN  NaN
11         NaN   NaN  NaN
12         NaN   NaN  NaN
13         NaN   NaN  NaN
14         NaN   NaN  NaN
15         NaN   NaN  NaN
16         NaN   NaN  NaN
17         NaN   NaN  NaN
18         NaN   NaN  NaN
19         NaN   NaN  NaN
20         NaN   NaN  NaN
21         NaN   NaN  NaN
22         NaN   NaN  NaN
23         NaN   NaN  NaN
24         NaN   NaN  NaN
25         NaN   NaN  NaN
26         NaN   NaN  NaN
27         NaN   NaN  NaN
28         NaN   NaN  NaN
29         NaN   NaN  NaN
...        ...   ...  ...
74262      NaN   NaN  NaN
74263      NaN   NaN  NaN
74264      NaN   NaN  NaN
74265      NaN   NaN  NaN
74266      NaN   NaN  NaN
74267      NaN   NaN  NaN
74268      N

In [26]:
from fuzzywuzzy import fuzz

#To be implemented. We will find how to combine similarity ratio of business name and zip code.
#Currently, we only consider similarity of business name.
def computeSimilarity(business_temp, master_temp):
    for index,row in business_temp.iterrows():
        name_similarity = 0
        zip_similarity = 0
        max_combine = 0
        master_name = "xxx"
        master_zip = 0
        business_name = row['business']
        business_zip = row['zip']

        for index1, row1 in master_temp.iterrows():
            business = row1['business']
            zipcode = row1['zip']
            name_similar = fuzz.ratio(business_name.lower(), business.lower())
            zip_similar = fuzz.ratio(business_zip, zipcode)

            if(zip_similar<40):
                combine_similar = name_similar
            else:
                combine_similar = name_similar * 0.8 + zip_similar * 0.2

            if(combine_similar > max_combine):
                name_similarity = name_similar
                zip_similarity = zip_similar
                max_combine = combine_similar
                master_name = business
                master_zip = zipcode
        business_temp.loc[index, 'name_match_percent'] = name_similarity
        business_temp.loc[index, 'zip_match_percent'] = zip_similarity
        business_temp.loc[index, 'combined_percent'] = max_combine
        business_temp.loc[index, 'master_businessname'] = master_name
        business_temp.loc[index, 'master_zip'] = master_zip

In [27]:
def markDuplicate(business_temp):
    for index, row in business_temp.iterrows():
        if(row['combined_percent']>85):
            business_temp.loc[index, 'duplicate?'] = 1
        else:
            business_temp.loc[index, 'duplicate?'] = 0

In [28]:
business_bus_st['zip_match_percent'] = pd.Series(0, index=business_bus_st.index)
business_bus_st['name_match_percent'] = pd.Series(0, index=business_bus_st.index)
business_bus_st['combined_percent'] = pd.Series(0, index=business_bus_st.index)
business_bus_st['master_businessname'] = pd.Series("", index=business_bus_st.index)
business_bus_st['master_zip'] = pd.Series(0, index=business_bus_st.index)
business_bus_st['duplicat?'] = pd.Series(0, index=business_bus_st.index)  

print (business_bus_st)

                                                business state    zip  \
0                                  A & A ROOFING CO, INC    AK  99707   
1                          A AND W WHOLESALE COMPANY INC    AK  99707   
2                     ELAINE S. BAKER & ASSOCIATES, INC.    AK  99501   
3                           DON ABEL BUILDING SUPPLY INC    AK  99801   
4                                 ALASKA AIRLINES, INC      WA  98168   
5                                 ALASKA FLOR-WALL, INC.    AK  99501   
6                                  FAIRBANKS GOLDPANNERS    AK  99707   
7                                 ALASKA INDIAN ARTS INC    AK  99827   
8                        ALASKA INDUSTRIAL HARDWARE, INC    AK  99501   
9                                       ALASKA LANDS INC    AK  99603   
10                                       AVIS RENT A CAR    AK  99519   
11                          ALASKA SAUSAGE COMPANY, INC.    AK  99509   
12              ALASKA VILLAGE ELECTRIC COOPERATIVE

In [29]:
#Separate dataframe by state to improve the performance.
#We will write a loop here to process for all states 

list_of_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
                 "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
                 "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"] 

business_temp_aggregate = pd.DataFrame(data=None, columns=business_bus_st.columns,index=[0])

In [30]:

print (business_temp_aggregate)


  business state  zip zip_match_percent name_match_percent combined_percent  \
0      NaN   NaN  NaN               NaN                NaN              NaN   

  master_businessname master_zip duplicat?  
0                 NaN        NaN       NaN  


In [ ]:
#for state in list_of_states: 


list_of_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
                 "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
                 "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",""] 

for state in list_of_states:
    master_temp = master_bus_st[(master_bus_st.state==state)]
    business_temp = business_bus_st[(business_bus_st.state==state)] 
    computeSimilarity(business_temp, master_temp)
    markDuplicate(business_temp) 
    business_temp_aggregate = business_temp_aggregate.append(business_temp)



c:\program files (x86)\python36-32\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
c:\program files (x86)\python36-32\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [ ]:
print (business_temp_aggregate)

In [28]:
#business_co[business_co['distance'] < 10].to_csv("compare.csv")

#We have a similarity percent of business name.
def exportcsv(business_temp):
    business_temp.to_csv("percent.csv")

In [24]:
computeSimilarity(business_co, master_co)

c:\users\yang\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [28]:
markDuplicate(business_co)
exportcsv(business_co)

c:\users\yang\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\yang\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
